Создайте папку data, а в ней папки page_texts и raw_pages

In [167]:
!pip install beautifulsoup4

In [2]:
import requests
import re
from bs4 import BeautifulSoup

In [168]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'

def get_page_text(url, user_agent, error_log=True):
    headers = {'User-Agent': user_agent}
    response = requests.get(url, headers=headers, allow_redirects=True)

    if response.status_code == 200:
        page_text = response.text
        return page_text
    elif error_log:
        print(f'Ошибка! Response code: {response.status_code}')
        print(f'URL: {url}')
        print('--------------')
    
    return None

In [185]:
index = 1
url = f'https://www.kinopoisk.ru/media/?page={index}'

page_text = get_page_text(url, USER_AGENT)

In [170]:
url

'https://www.kinopoisk.ru/media/?page=1'

In [186]:
print(page_text[:1000])

<!DOCTYPE html>
<html lang="ru" class="ua_js_no app-theme_light">
    <head>
        <title>Медиа на КиноПоиске</title>
        <meta charset="utf-8" />
        <meta name="description" content="Медиа на КиноПоиске. Самая свежая и интересная информация из мира кино: новости, статьи о звездах, интервью, репортажи, обзоры премьер и кассовых сборов." />
        <meta name="keywords" content="" />
        <meta name="apple-mobile-web-app-status-bar-style" content="#444444">
        <meta name="apple-mobile-web-app-title" content="КиноПоиск">
        <meta name="apple-itunes-app" content="app-id=477718890, ct=kp-web, pt=214944, mt=8">

        <meta name="viewport" content="width=device-width" />
        <meta property="og:title" content="Медиа на КиноПоиске" />
        <meta property="og:description" content="Медиа на КиноПоиске. Самая свежая и интересная информация из мира кино: новости, статьи о звездах, интервью, репортажи, обзоры премьер и кассовых сборов." />
        <meta property="o

In [187]:
from os import makedirs

soup = BeautifulSoup(page_text)
posts_list = soup.find_all('div', {'class': 'posts-grid__main-section-column'})
for p in posts_list:
    lnk = p.find('a').attrs['href']
    title = re.sub('[^А-ЯЁа-яё0-9\s]', ' ', p.text)
    title = re.sub('\s\s+', ' ', title)
    
    page_url = 'https://www.kinopoisk.ru' + lnk
    clean_path = '/'.join([d for d in page_url.split('/')[2:] if len(d) > 0])
    
    page_text = get_page_text(page_url, USER_AGENT)
    
    if page_text is None:
        continue
        
    dir_path = 'data/raw_pages/' + '/'.join(clean_path.split('/')[:-1])
    makedirs(dir_path, exist_ok=True)
    with open(dir_path + '/' + clean_path.split('/')[-1] + '.html', 'w', encoding='utf-8') as f:
        f.write(page_text)
            
#     print(page_url, title)

In [188]:
main_url = f'https://www.kinopoisk.ru/'

page_text = get_page_text(main_url, USER_AGENT)
soup = BeautifulSoup(page_text)

In [115]:
def clear_link(page_link):
    page_link = re.sub('^\/', '', page_link)
    page_link = re.sub('[\#\?].+', '', page_link)
    
    return page_link

In [194]:
links = set()
for a in soup.find_all('a'):
    if not a.has_attr('href'):
        continue
    
    page_link = a.attrs['href']
    
    if re.search('^(http)', page_link):
        if main_url not in page_link or main_url == page_link:
            continue
    
    page_link = clear_link(page_link)
    
    if len(page_link) == 0:
        continue 
    
    page_link = main_url + page_link
    
    links.add(page_link)
#     print(page_link)

links

{'https://www.kinopoisk.ru/afisha/new/',
 'https://www.kinopoisk.ru/afisha/new/city/',
 'https://www.kinopoisk.ru/afisha/new/film/1045449/',
 'https://www.kinopoisk.ru/afisha/new/film/1046057/',
 'https://www.kinopoisk.ru/afisha/new/film/1199100/',
 'https://www.kinopoisk.ru/afisha/new/film/1219003/',
 'https://www.kinopoisk.ru/afisha/new/film/1219149/',
 'https://www.kinopoisk.ru/afisha/new/film/1227967/',
 'https://www.kinopoisk.ru/afisha/new/film/1243567/',
 'https://www.kinopoisk.ru/afisha/new/film/1264209/',
 'https://www.kinopoisk.ru/afisha/new/film/1264778/',
 'https://www.kinopoisk.ru/afisha/new/film/1320744/',
 'https://www.kinopoisk.ru/afisha/new/film/1322857/',
 'https://www.kinopoisk.ru/afisha/new/film/1356128/',
 'https://www.kinopoisk.ru/afisha/new/film/1378456/',
 'https://www.kinopoisk.ru/afisha/new/film/1398967/',
 'https://www.kinopoisk.ru/afisha/new/film/1437864/',
 'https://www.kinopoisk.ru/afisha/new/film/1449918/',
 'https://www.kinopoisk.ru/afisha/new/film/409424

In [130]:
def get_page_links(soup, main_url):
    links = set()
    for a in soup.find_all('a'):
        if not a.has_attr('href'):
            continue

        page_link = a.attrs['href']

        if re.search('^(http)', page_link):
            if main_url not in page_link or main_url == page_link:
                continue

        page_link = clear_link(page_link)

        if len(page_link) == 0:
            continue 

        page_link = main_url + page_link

        links.add(page_link)
    
    return links

In [131]:
get_page_links(soup, main_url)

{'https://www.kinopoisk.ru/afisha/new/',
 'https://www.kinopoisk.ru/box/',
 'https://www.kinopoisk.ru/box/usa/',
 'https://www.kinopoisk.ru/box/world/',
 'https://www.kinopoisk.ru/comingsoon/digital/',
 'https://www.kinopoisk.ru/docs/usage/',
 'https://www.kinopoisk.ru/film/1045449/',
 'https://www.kinopoisk.ru/film/1199100/',
 'https://www.kinopoisk.ru/film/1219149/',
 'https://www.kinopoisk.ru/film/1227967/',
 'https://www.kinopoisk.ru/film/1243567/',
 'https://www.kinopoisk.ru/film/1244460/',
 'https://www.kinopoisk.ru/film/1253400/',
 'https://www.kinopoisk.ru/film/1264209/',
 'https://www.kinopoisk.ru/film/1301966/',
 'https://www.kinopoisk.ru/film/1303057/',
 'https://www.kinopoisk.ru/film/1309482/',
 'https://www.kinopoisk.ru/film/1309570/',
 'https://www.kinopoisk.ru/film/1309758/',
 'https://www.kinopoisk.ru/film/1322389/',
 'https://www.kinopoisk.ru/film/1322389/watch/',
 'https://www.kinopoisk.ru/film/1322857/',
 'https://www.kinopoisk.ru/film/1322912/',
 'https://www.kinopo

In [195]:
# Сбор всех ссылок с ресурса
main_url = f'https://www.kinopoisk.ru/'

page_text = get_page_text(main_url, USER_AGENT)
soup = BeautifulSoup(page_text)

all_links = get_page_links(soup, main_url)
process_links = all_links

i = 0 # Для мастер-класса
while True:
    new_links = set()

    for lnk in process_links:
        page_text = get_page_text(lnk, USER_AGENT)
        
        if 
        soup = BeautifulSoup(page_text)
        links = get_page_links(soup, main_url)
        links = links.difference(all_links, new_links)

        new_links = new_links.union(links)

    if len(new_links) == 0:
        break

    all_links = all_links.union(new_links)
    process_links = new_links
    
    # Для мастер-класса
    i += 1
    if i == 10:
        break

Ошибка! Response code: 404
URL: https://www.kinopoisk.ru/https://www.kinopoisk.ru/media/game/4000802/
--------------


TypeError: object of type 'NoneType' has no len()

In [197]:
len(all_links)

1976